# Using pyspark Data-PreProcessing


In [1]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'

In [2]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

In [3]:
import pyspark.sql.functions as F
from pyspark.sql.types import *

In [4]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DoubleType, DateType

city_df = spark.read.csv('file:///home/talentum/test-jupyter/Daily/program/Indian_Cities_Database.csv', header=True)

#for city in city_df.collect():
print(city["City"])

humid_df = spark.read.csv('file:///home/talentum/test-jupyter/Hourly/Adilabad_humid/*.csv', header=True)
temp_df = spark.read.csv('file:///home/talentum/test-jupyter/Hourly/Adilabad_temp/*.csv', header=True)
wind_df = spark.read.csv('file:///home/talentum/test-jupyter/Hourly/Adilabad_wind/*.csv', header=True)

# joining humid_df, temp_df and wind_df to produce final dataframe
final_df = humid_df.join(temp_df, humid_df.Date == temp_df.Date, 'inner') \
.join(wind_df , humid_df.Date == wind_df.Date, 'inner') \
.select(humid_df.Date, "PS","PSC","T2M","T2MWET","T2MDEW","WS2M","WD2M","WD10M","WS10M","QV2M","RH2M","PRECTOTCORR")

# final_df.show()
# write dataframe to file in csv format
final_df.write.csv(f'file:///home/talentum/test-jupyter/Hourly/humid/Alidabad',header=True)